In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

def load_images_from_folder(folder):
    images, labels = [], []
    label_dict = {'audi': 0, 'lamborghini': 1, 'mercedes': 2}

    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        if os.path.isdir(class_path):
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                try:
                    with Image.open(image_path) as img:
                        img = img.resize((150, 150))
                        img = img.convert('RGB')
                        images.append(np.array(img))
                        labels.append(label_dict[class_folder])
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")

    return np.array(images), np.array(labels)

# Load the training and testing data
train_dir = 'C:/Users/kh.sc.i5mca21162/Desktop/python/dataset/Images/Train'
test_dir = 'C:/Users/kh.sc.i5mca21162/Desktop/python/dataset/Images/Test'

train_images, train_labels = load_images_from_folder(train_dir)
test_images, test_labels = load_images_from_folder(test_dir)

print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

# Normalize pixel values
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Convert labels to categorical (one hot encoding)
train_labels = to_categorical(train_labels, 3)
test_labels = to_categorical(test_labels, 3)

# Create the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
for layer in base_model.layers:
    layer.trainable = False

# Build the model
model = Sequential()
model.add(InputLayer(input_shape=(150, 150, 3)))
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

# Calculate steps_per_epoch
steps_per_epoch = train_images.shape[0] // 32  # 32 is the batch size

# Train the model
history = model.fit(
    datagen.flow(train_images, train_labels, batch_size=32),
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=(test_images, test_labels)
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=1)
print(f'Test accuracy: {test_acc:.4f}')

# Calculate additional metrics
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

# Generate a classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=list(label_dict.keys())))

# Confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(label_dict))
plt.xticks


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/kh.sc.i5mca21162/Desktop/python/dataset/Images/Train'